In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

tf.random.set_seed(7)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
data_path = '/content/drive/MyDrive/data/planeta'
train = pd.read_csv(os.path.join(data_path, 'train_data.csv'))
train.head(10)

,TIO2(WT%),AL2O3(WT%),FEOT(WT%),CAO(WT%),MGO(WT%),MNO(WT%),K2O(WT%),NA2O(WT%),P2O5(WT%),SC(PPM),V(PPM),CR(PPM),CO(PPM),NI(PPM),CU(PPM),ZN(PPM),RB(PPM),SR(PPM),Y(PPM),ZR(PPM),NB(PPM),CS(PPM),BA(PPM),LA(PPM),CE(PPM),PR(PPM),ND(PPM),SM(PPM),EU(PPM),GD(PPM),TB(PPM),DY(PPM),HO(PPM),ER(PPM),TM(PPM),YB(PPM),LU(PPM),HF(PPM),TA(PPM),PB(PPM),TH(PPM),U(PPM),Thickness
0,0.560000,16.000000,6.703510,6.600000,5.100000,0.11000,1.300000,3.610000,0.150000,18.000000,177.000,165.00000,33.00,81.0,NaN,NaN,29.000000,505.000000,10.800000,81.000000,3.000000,NaN,550.000000,12.100000,25.000000,NaN,13.000000,2.600000,0.770000,2.40000,NaN,2.000000,NaN,1.10000,NaN,0.990000,NaN,NaN,NaN,NaN,2.650000,NaN,0
1,1.130000,17.210000,8.886916,12.120000,7.680000,0.15000,0.240000,2.700000,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.530000,16.150000,2.996334,3.980000,1.990000,0.06000,2.180000,4.580000,0.180000,8.050000,68.700,47.80000,6.38,10.8,6.49000,80.40000,65.200000,756.000000,11.500000,161.000000,5.650000,2.140000,1521.000000,24.800000,47.500000,5.92000,22.700000,4.230000,1.110000,3.50000,0.461000,2.400000,0.429000,1.13000,0.156000,1.000000,0.142000,NaN,0.339000,16.700000,7.000000,2.350000,0
3,1.150000,15.100000,7.891246,7.740000,7.930000,0.12000,1.670000,3.010000,0.270000,24.050000,NaN,NaN,NaN,NaN,NaN,NaN,47.000000,490.000000,18.830000,172.000000,12.500000,1.310000,409.000000,35.300000,67.900000,7.51000,30.300000,5.720000,1.390000,5.20000,0.660000,3.840000,0.700000,1.98000,0.300000,1.880000,0.260000,6.740000,0.780000,9.100000,7.600000,1.200000,0
4,0.420000,15.950000,3.761164,4.090000,1.900000,0.06000,2.470000,4.420000,0.140000,7.000000,54.000,32.00000,11.00,16.0,NaN,NaN,68.000000,555.000000,7.000000,28.000000,3.600000,NaN,935.000000,23.000000,43.000000,NaN,18.500000,3.400000,0.900000,2.40000,NaN,1.350000,NaN,0.60000,NaN,0.500000,NaN,NaN,NaN,NaN,5.100000,NaN,0
5,0.370000,15.600000,2.166834,1.720000,1.370000,0.05000,4.320000,3.180000,0.240000,7.000000,58.000,61.00000,7.30,22.0,24.00000,87.00000,373.000000,195.000000,19.000000,152.000000,12.000000,105.000000,820.000000,40.400000,80.600000,NaN,35.100000,4.900000,1.040000,NaN,0.340000,NaN,NaN,NaN,NaN,1.310000,0.190000,5.000000,2.900000,62.000000,22.200000,14.600000,0
6,0.650000,16.500000,5.128860,5.000000,2.640000,0.09000,2.420000,4.140000,0.200000,10.900000,127.000,32.00000,16.50,27.0,NaN,NaN,47.500000,680.000000,12.000000,120.000000,5.500000,NaN,990.000000,25.000000,45.000000,NaN,20.000000,3.700000,1.030000,3.35000,NaN,2.150000,NaN,1.10000,NaN,1.040000,NaN,NaN,NaN,NaN,3.000000,NaN,0
7,0.560000,16.800000,5.173850,5.980000,3.450000,0.10000,1.370000,3.710000,0.160000,14.000000,120.000,112.00000,18.00,33.0,NaN,NaN,26.000000,510.000000,10.400000,78.000000,4.000000,NaN,600.000000,12.200000,23.500000,NaN,13.500000,NaN,0.770000,NaN,NaN,1.900000,NaN,1.10000,NaN,0.950000,NaN,NaN,NaN,NaN,2.500000,NaN,0
8,1.618579,13.456672,6.442568,13.126606,10.654914,0.20247,1.326228,2.703669,0.916604,29.733778,302.514,468.36075,NaN,NaN,188.47575,109.07575,83.142693,1455.172778,27.738894,164.805672,22.120503,1.006771,1016.289592,57.655516,121.236541,15.17592,61.428078,11.834762,3.390597,9.15931,1.222219,6.209152,1.108197,2.65357,0.347112,1.996277,0.288843,4.132123,1.194383,7.891174,6.963459,2.021621,0
9,0.750000,17.650000,6.226616,6.150000,2.980000,0.12000,2.090000,4.480000,0.370000,10.700000,144.000,26.00000,21.00,25.0,NaN,NaN,51.800000,890.000000,14.000000,155.000000,9.000000,NaN,1230.000000,31.500000,59.000000,NaN,26.500000,5.100000,1.390000,3.90000,NaN,2.500000,NaN,1.25000,NaN,1.190000,NaN,NaN,NaN,NaN,5.650000,NaN,0


In [21]:
y = train['Thickness']
train = train.drop(['Thickness'], axis=1)

In [22]:
test = pd.read_csv(os.path.join(data_path,'test_input.csv'))
test.head(10)

,TIO2(WT%),AL2O3(WT%),FEOT(WT%),CAO(WT%),MGO(WT%),MNO(WT%),K2O(WT%),NA2O(WT%),P2O5(WT%),SC(PPM),V(PPM),CR(PPM),CO(PPM),NI(PPM),CU(PPM),ZN(PPM),RB(PPM),SR(PPM),Y(PPM),ZR(PPM),NB(PPM),CS(PPM),BA(PPM),LA(PPM),CE(PPM),PR(PPM),ND(PPM),SM(PPM),EU(PPM),GD(PPM),TB(PPM),DY(PPM),HO(PPM),ER(PPM),TM(PPM),YB(PPM),LU(PPM),HF(PPM),TA(PPM),PB(PPM),TH(PPM),U(PPM)
0,1.000000,15.340000,8.740000,13.300000,7.680000,0.14000,0.260000,2.330000,0.090000,39.419761,264.235040,298.191685,40.741070,102.035872,93.808322,68.583517,5.111586,148.980391,22.755959,50.203352,1.570279,0.128355,36.125974,3.587330,9.172711,1.455107,7.418125,2.517975,0.957632,3.032573,0.521706,3.445595,0.753475,2.175624,0.329385,2.089226,0.324914,1.407370,0.108011,0.843640,0.305881,0.115163
1,1.105920,14.375060,11.438840,12.361560,7.166060,0.20634,0.046540,2.457160,0.073120,44.178022,333.120353,61.324963,49.626789,59.147545,130.139826,103.477341,0.533042,63.740163,26.635604,51.645219,0.706426,0.005765,4.860177,1.438734,5.181906,0.965136,5.679726,2.270337,0.896736,3.609905,0.691708,4.625839,1.021353,2.987150,0.442955,2.974130,0.449079,1.414753,0.044261,0.377547,0.074079,0.024858
2,0.570000,17.100000,9.699922,4.530000,2.550000,0.09000,1.130000,4.530000,0.050000,11.000000,106.000000,75.000000,18.336469,27.000000,33.717415,78.814423,8.900000,452.000000,8.800000,132.000000,3.700000,0.440331,660.000000,8.800000,20.500000,2.344961,9.700000,1.900000,0.820000,1.950000,0.269828,1.600000,0.311089,0.850000,0.123287,0.880000,0.129984,2.473764,0.259204,5.950845,2.200000,0.806990
3,1.310000,15.990000,8.043628,10.960000,6.700000,0.16000,0.480000,2.940000,0.170000,35.200000,252.000000,109.000000,32.700000,52.300000,48.900000,65.800000,6.240000,200.000000,26.500000,91.800000,4.300000,0.320000,52.600000,5.620000,14.200000,2.170000,11.400000,3.490000,1.290000,3.858417,0.770000,5.000000,1.060000,3.090000,0.480000,3.040000,0.440000,2.710000,0.330000,2.520000,0.450000,0.190000
4,1.400000,17.980000,9.780000,9.110000,3.280000,0.17000,1.660000,3.330000,0.320000,24.720000,225.000000,30.100000,26.000000,8.000000,64.853091,113.980633,31.000000,935.000000,17.000000,94.000000,9.000000,1.100000,592.000000,25.400000,51.000000,6.138364,26.000000,5.780000,1.990000,5.496627,0.800000,4.293143,0.841318,2.408336,0.383675,2.400000,0.400000,2.900000,0.500000,6.000000,3.400000,0.700000
5,0.610000,16.300000,6.130000,5.780000,3.740000,0.10000,1.050000,4.220000,0.150000,19.000000,140.000000,87.000000,19.804545,29.000000,31.000000,80.000000,15.700000,424.000000,12.100000,80.000000,2.750000,0.590000,449.000000,7.400000,15.200000,2.040000,9.440000,2.520000,0.810000,2.550000,0.390000,2.230000,0.450000,1.180000,0.170000,1.030000,0.170000,2.300000,0.180000,4.480000,1.050000,0.480000
6,1.700000,14.400000,12.210000,7.900000,3.520000,0.20000,0.450000,2.860000,0.260000,30.400000,399.722965,0.596000,31.100000,2.250000,48.400000,91.300000,6.850000,174.000000,28.900000,62.000000,0.826000,0.169000,93.500000,3.800000,10.400000,1.740000,9.100000,3.120000,1.160000,4.080000,0.750000,5.090000,1.090000,3.160000,0.471000,3.200000,0.484000,1.930000,0.058000,1.220000,0.356000,0.378000
7,0.709644,16.911526,5.288174,5.617185,3.358317,0.07996,1.489254,4.047972,0.189905,14.858667,139.557333,96.381333,18.757667,52.995333,31.024000,150.827667,29.161333,574.000000,10.069667,103.901667,4.379667,0.392000,652.204667,12.521667,24.905667,3.234667,13.514667,3.014333,0.904333,2.444000,0.354667,1.977000,0.359333,0.911000,0.140000,0.911000,0.131000,2.693333,0.279000,0.081000,4.513333,2.226667
8,0.500000,15.050000,2.508738,2.450000,1.060000,0.05000,4.080000,4.310000,0.170000,0.000000,56.000000,8.000000,8.000000,4.000000,9.000000,64.000000,121.000000,481.000000,14.000000,211.000000,11.000000,3.130000,1074.000000,23.900000,60.100000,5.630000,19.900000,3.360000,0.813000,3.020000,0.344000,1.780000,0.316000,0.915000,0.128000,0.813000,0.131000,4.170000,0.604000,14.000000,10.400000,1.810000
9,0.180000,10.470000,7.765592,6.900000,14.180000,0.14000,0.720000,0.600000,0.050000

In [23]:
train = train.interpolate()

In [24]:
#train['CO(PPM)'] = np.where(train['CO(PPM)'] == 0, train['SC(PPM)'],  train['CO(PPM)'])
#train['SC(PPM)'] = np.where(train['SC(PPM)'] == 0, train['CO(PPM)'],  train['SC(PPM)'])

In [25]:
'''
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

X = np.array(train)

# 결측치 평균값으로 채우기
imp_mean = SimpleImputer(missing_values=0, strategy='mean')
# imp_mean = IterativeImputer(missing_values=0)

train = pd.DataFrame(imp_mean.fit_transform(X), columns=train.columns)
'''

"\nimport numpy as np\nfrom sklearn.experimental import enable_iterative_imputer\nfrom sklearn.impute import IterativeImputer, SimpleImputer\n\nX = np.array(train)\n\n# 결측치 평균값으로 채우기\nimp_mean = SimpleImputer(missing_values=0, strategy='mean')\n# imp_mean = IterativeImputer(missing_values=0)\n\ntrain = pd.DataFrame(imp_mean.fit_transform(X), columns=train.columns)\n"

In [26]:
train = train.fillna(0)

In [27]:
for i in train.columns:
  train[i] = np.where(train[i] == 0, train[i].mean(),  train[i])

In [28]:
col_list = train.columns.tolist()
col_train = train[col_list]
col_test = test[col_list]

train['sum'] = col_train.sum(axis=1)
test['sum'] = col_test.sum(axis=1)

In [29]:
train['alcali'] = [1 if mgo > 7 else 0 for mgo in train['MGO(WT%)']]
test['alcali'] = [1 if mgo > 7 else 0 for mgo in test['MGO(WT%)']]

In [30]:
#train = (train - train.mean())/train.std()
#test = (test - test.mean())/test.std()

In [31]:
display(train)

,TIO2(WT%),AL2O3(WT%),FEOT(WT%),CAO(WT%),MGO(WT%),MNO(WT%),K2O(WT%),NA2O(WT%),P2O5(WT%),SC(PPM),V(PPM),CR(PPM),CO(PPM),NI(PPM),CU(PPM),ZN(PPM),RB(PPM),SR(PPM),Y(PPM),ZR(PPM),NB(PPM),CS(PPM),BA(PPM),LA(PPM),CE(PPM),PR(PPM),ND(PPM),SM(PPM),EU(PPM),GD(PPM),TB(PPM),DY(PPM),HO(PPM),ER(PPM),TM(PPM),YB(PPM),LU(PPM),HF(PPM),TA(PPM),PB(PPM),TH(PPM),U(PPM),sum,alcali
0,0.560,16.000,6.703510,6.600,5.100,0.110,1.300,3.61,0.150,18.000000,177.000000,165.000000,33.000000,81.000000,74.067720,97.961523,29.00000,505.000000,10.800000,81.000000,3.000000,7.818735,550.000000,12.100000,25.000000,5.553202,13.000000,2.600000,0.770000,2.400000,0.693047,2.000000,0.713709,1.100000,0.283545,0.990000,0.291324,3.627104,0.969755,11.929243,2.650000,3.283823,1962.736239,0
1,1.130,17.210,8.886916,12.120,7.680,0.150,0.240,2.70,0.100,13.025000,122.850000,106.400000,19.690000,45.900000,74.067720,97.961523,47.10000,630.500000,11.150000,121.000000,4.325000,7.818735,1035.500000,18.450000,36.250000,5.553202,17.850000,3.415000,0.940000,2.950000,0.693047,2.200000,0.713709,1.115000,0.283545,0.995000,0.291324,3.627104,0.969755,11.929243,4.825000,3.283823,2503.839645,1
2,0.530,16.150,2.996334,3.980,1.990,0.060,2.180,4.58,0.180,8.050000,68.700000,47.800000,6.380000,10.800000,6.490000,80.400000,65.20000,756.000000,11.500000,161.000000,5.650000,2.140000,1521.000000,24.800000,47.500000,5.920000,22.700000,4.230000,1.110000,3.500000,0.461000,2.400000,0.429000,1.130000,0.156000,1.000000,0.142000,3.627104,0.339000,16.700000,7.000000,2.350000,2929.250438,0
3,1.150,15.100,7.891246,7.740,7.930,0.120,1.670,3.01,0.270,24.050000,61.350000,39.900000,8.690000,13.400000,12.326667,82.600000,47.00000,490.000000,18.830000,172.000000,12.500000,1.310000,409.000000,35.300000,67.900000,7.510000,30.300000,5.720000,1.390000,5.200000,0.660000,3.840000,0.700000,1.980000,0.300000,1.880000,0.260000,6.740000,0.780000,9.100000,7.600000,1.200000,1626.197913,1
4,0.420,15.950,3.761164,4.090,1.900,0.060,2.470,4.42,0.140,7.000000,54.000000,32.000000,11.000000,16.000000,18.163333,84.800000,68.00000,555.000000,7.000000,28.000000,3.600000,53.155000,935.000000,23.000000,43.000000,9.043184,18.500000,3.400000,0.900000,2.400000,0.500000,1.350000,0.781639,0.600000,0.309422,0.500000,0.225000,5.870000,1.840000,35.550000,5.100000,7.900000,2066.698743,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7622,0.530,11.750,9.528882,12.540,9.940,0.160,2.740,1.33,0.370,44.400000,239.000000,363.000000,44.500000,78.000000,39.200000,81.000000,47.00000,1169.000000,15.000000,40.000000,4.733333,1.090000,672.000000,7.600000,15.800000,4.400000,9.600000,2.880000,0.990000,2.480717,0.530000,2.147207,0.400000,1.078629,0.200000,1.400000,0.200000,1.020000,0.030000,5.000000,0.910000,0.600000,2944.078768,1
7623,1.690,16.190,6.800000,9.990,7.630,0.110,1.700,3.83,0.810,34.591573,267.500000,221.500000,51.250000,65.000000,44.400000,83.000000,26.20000,681.000000,23.000000,78.000000,2.366667,1.090000,355.000000,7.195000,16.450000,4.400000,11.510000,3.450000,1.220000,2.461433,0.685000,2.194413,0.400000,1.057258,0.200000,2.330000,0.341000,1.020000,0.030000,6.000000,1.013537,0.600000,2045.205881,1
7624,1.510,16.370,8.250000,9.740,5.430,0.200,0.400,3.44,0.220,24.783146,296.000000,80.000000,58.000000,52.000000,49.600000,85.000000,5.40000,193.000000,31.000000,116.000000,9.656184,1.090000,38.000000,6.790000,17.100000,4.400000,13.420000,4.020000,1.450000,2.442150,0.840000,2.241620,0.400000,1.035887,0.200000,3.260000,0.482000,1.020000,0.030000,7.000000,1.117074,0.600000,1152.938061,0
7625,0.590,17.920,6.514552,6.990,3.170,0.120,1.110,3.81,0.160,14.974719,144.794748,30.719457,18.173772,15.351869,54.800000,87.000000,12.61644,513.970652,12.706569,88.982648,3.697229,1.090000,517.378137,9.608943,17.739124,4.400000,10.808273,2.650903,0.704437,2.422866,0.840000,2.288827,0.400000,1.014516,0.200000,1.196026,0.482000,1.020000,0.030000,8.0

In [32]:
display(test)

,TIO2(WT%),AL2O3(WT%),FEOT(WT%),CAO(WT%),MGO(WT%),MNO(WT%),K2O(WT%),NA2O(WT%),P2O5(WT%),SC(PPM),V(PPM),CR(PPM),CO(PPM),NI(PPM),CU(PPM),ZN(PPM),RB(PPM),SR(PPM),Y(PPM),ZR(PPM),NB(PPM),CS(PPM),BA(PPM),LA(PPM),CE(PPM),PR(PPM),ND(PPM),SM(PPM),EU(PPM),GD(PPM),TB(PPM),DY(PPM),HO(PPM),ER(PPM),TM(PPM),YB(PPM),LU(PPM),HF(PPM),TA(PPM),PB(PPM),TH(PPM),U(PPM),sum,alcali
0,1.00000,15.34000,8.740000,13.30000,7.68000,0.14000,0.26000,2.33000,0.09000,39.419761,264.235040,298.191685,40.741070,102.035872,93.808322,68.583517,5.111586,148.980391,22.755959,50.203352,1.570279,0.128355,36.125974,3.587330,9.172711,1.455107,7.418125,2.517975,0.957632,3.032573,0.521706,3.445595,0.753475,2.175624,0.329385,2.089226,0.324914,1.407370,0.108011,0.843640,0.305881,0.115163,1261.332604,1
1,1.10592,14.37506,11.438840,12.36156,7.16606,0.20634,0.04654,2.45716,0.07312,44.178022,333.120353,61.324963,49.626789,59.147545,130.139826,103.477341,0.533042,63.740163,26.635604,51.645219,0.706426,0.005765,4.860177,1.438734,5.181906,0.965136,5.679726,2.270337,0.896736,3.609905,0.691708,4.625839,1.021353,2.987150,0.442955,2.974130,0.449079,1.414753,0.044261,0.377547,0.074079,0.024858,1013.542028,1
2,0.57000,17.10000,9.699922,4.53000,2.55000,0.09000,1.13000,4.53000,0.05000,11.000000,106.000000,75.000000,18.336469,27.000000,33.717415,78.814423,8.900000,452.000000,8.800000,132.000000,3.700000,0.440331,660.000000,8.800000,20.500000,2.344961,9.700000,1.900000,0.820000,1.950000,0.269828,1.600000,0.311089,0.850000,0.123287,0.880000,0.129984,2.473764,0.259204,5.950845,2.200000,0.806990,1717.828511,0
3,1.31000,15.99000,8.043628,10.96000,6.70000,0.16000,0.48000,2.94000,0.17000,35.200000,252.000000,109.000000,32.700000,52.300000,48.900000,65.800000,6.240000,200.000000,26.500000,91.800000,4.300000,0.320000,52.600000,5.620000,14.200000,2.170000,11.400000,3.490000,1.290000,3.858417,0.770000,5.000000,1.060000,3.090000,0.480000,3.040000,0.440000,2.710000,0.330000,2.520000,0.450000,0.190000,1086.522045,0
4,1.40000,17.98000,9.780000,9.11000,3.28000,0.17000,1.66000,3.33000,0.32000,24.720000,225.000000,30.100000,26.000000,8.000000,64.853091,113.980633,31.000000,935.000000,17.000000,94.000000,9.000000,1.100000,592.000000,25.400000,51.000000,6.138364,26.000000,5.780000,1.990000,5.496627,0.800000,4.293143,0.841318,2.408336,0.383675,2.400000,0.400000,2.900000,0.500000,6.000000,3.400000,0.700000,2365.615187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,0.69000,16.95000,5.083870,5.20000,2.30000,0.09000,2.36000,4.15000,0.28000,9.500000,112.000000,18.000000,15.000000,16.000000,46.958837,80.486079,47.000000,810.000000,12.200000,172.000000,6.900000,1.135212,1080.000000,34.500000,65.000000,7.600183,29.500000,4.633958,1.260000,3.450670,0.456055,2.200000,0.433886,1.300000,0.172841,1.030000,0.159228,4.141983,0.494936,11.236254,3.400000,0.716591,2635.970584,0
2275,1.81000,16.70000,7.576316,6.88000,4.27000,0.09000,2.26000,4.23000,0.75000,3.000000,62.000000,1.000000,1.000000,2.000000,20.000000,37.000000,235.898000,20.797000,18.312000,89.982000,21.559000,6.400000,47.700000,25.442000,56.500000,6.069000,20.949000,3.996000,0.262000,3.208000,0.437000,2.759000,0.526000,1.428000,0.276000,1.784000,0.261000,3.129000,2.372000,20.585000,37.332000,8.223000,806.752316,0
2276,0.80000,14.60000,5.700000,3.60000,1.20000,0.16000,2.80000,3.60000,0.28000,17.000000,39.248826,1.000000,9.000000,1.000000,23.000000,92.750800,51.000000,222.000000,39.000000,208.000000,7.000000,1.700000,451.000000,15.480000,34.530000,4.900000,22.240000,5.950000,1.480000,6.500000,1.070000,6.920000,1.440000,4.310000,0.640000,4.310000,0.640000,5.530000,0.524752,6.600000,2.600000,1.400000,1322.504378,0
2277,2.65000,14.10000,11.060000,5.70000,1.80000,0.22000,2.40000,3.40000,0.64000,19.766414,215.934803,57.331893,30.571087,31.335708,52.837793,248.508029,76.846510,815.428355,31.281400,181.783300,23.875300,4.855954,938.798406,2

In [33]:
X = train.copy()

In [34]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(6101, 44)
(1526, 44)
(6101,)
(1526,)


In [35]:
# Random Forest 모델로 학습
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)
y_pred=random_forest.predict(X_val)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       953
           1       0.97      0.99      0.98       573

    accuracy                           0.98      1526
   macro avg       0.98      0.99      0.98      1526
weighted avg       0.99      0.98      0.98      1526



In [36]:
y_pred = random_forest.predict(test)
test['Thickness'] = y_pred
submission = test['Thickness']
submission.to_csv(os.path.join(data_path, "submission.csv"), index=False)
display(submission)

0       1
1       1
2       0
3       1
4       1
       ..
2274    0
2275    0
2276    1
2277    1
2278    0
Name: Thickness, Length: 2279, dtype: int64